<a href="https://colab.research.google.com/github/prashant29k/Optimization-of-Freezer-Utilization-for-Interchange-Group-Inc/blob/main/Optimization_of_Freezer_Utilization_%7C_Interchange_Group_Inc_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Interchange Group inc.

Install amplpy and other packages

In [ ]:
!pip install -q amplpy ampltools pandas

In [ ]:
# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "highs", "gokestrel"], "45550579-96ab-4901-aa14-3ef027ee3844"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics
import pandas as pd
import numpy as np

# instantiate AMPL object and register magics
ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

# # Initialize AMPL
# ampl = AMPL(Environment())

register_magics(ampl_object=ampl)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load the Data into AMPL

In [ ]:
SKU = pd.read_excel(
    "/content/drive/MyDrive/Colab Notebooks/FoodSupply.xlsx", sheet_name="SKU", index_col=0
)

remaining = pd.read_excel(
    "/content/drive/MyDrive/Colab Notebooks/FoodSupply.xlsx", sheet_name="remaining", index_col=0
)

inbound = pd.read_excel(
    "/content/drive/MyDrive/Colab Notebooks/FoodSupply.xlsx", sheet_name="incoming"
)
SKU = dict(zip(SKU.index, SKU.Hours))
remaining  = dict(zip(remaining.index, remaining['Remaining Time (Hrs)']))
sorted_temp = sorted([SKU[i] for i in inbound.Item if i in SKU.keys()], reverse=True)
incoming = dict([(i,j) for i,j in zip(range(len(sorted_temp)),sorted_temp)])

# Read and define data.

In [ ]:
ampl.eval ('''

reset;
set SKU;
set cells;
set pallets;

param time_taken {i in SKU};
param remaining {i in cells};
param pallet {i in pallets} ;

var x {i in cells, j in pallets} >= 0 binary;

minimize freeze_time {i in cells} : sum{j in pallets} (x[i, j] * pallet[j] )+ remaining[i];

subject to 
    each_row_pallets {i in cells}: sum {j in pallets} x[i, j]<= 1;
    first {i in cells} : sum{j in pallets} (x[i, j] * pallet[j] ) >= 1;                                           
''')

# Provide data to the model.

In [ ]:
ampl.set['SKU'] = SKU
ampl.set['cells'] = remaining
ampl.set['pallets'] = incoming

ampl.param['time_taken'] = SKU
ampl.param['remaining'] = remaining
ampl.param['pallet'] = incoming

#Set solver and solve.

In [ ]:
ampl.setOption('solver', 'gurobi')
ampl.solve()

Display problem formulation.

In [ ]:
ampl.display('freeze_time');

In [ ]:
fz=ampl.get_objective('freeze_time');
df_val = fz.get_values().to_pandas()
print(df_val)

In [ ]:
total_freeze = max(df_val.values.reshape(1, -1)[0])

Print solution and results.

In [ ]:
obj = ampl.get_objective('freeze_time')
print("\n")
#print("TotalCost is: ", obj.get().value(), "\n")
print("Optimal Allocation:")
ampl.display('x');

In [ ]:
var_x = ampl.get_variable('x').get_values().to_dict()
cells = set([i for i,j in var_x.keys()])
pallets = set([j for i,j in var_x.keys()])
data = pd.DataFrame([[var_x[(j, i)] for j in cells] for i in pallets], columns= list(cells))
cells_temp = dict(zip(df_val.index, df_val.values))

# Allocation

In [ ]:
for i,j in var_x:
  if var_x[(i,j)] > 0:
    print(f'{i} <---- Total time: {cells_temp[i][0]:.2f} \t Remaining Time: {remaining[i]:.2f} \t Pallet Time: {incoming[j]}')

# Total Freeze time

In [ ]:
print(f'The optimal solution is for interchange Group inc. is to have {total_freeze} hours of freezing time.') 